In [2]:
import numpy as np

import tensorflow as tf

In [14]:
def generate_weights(coefficients, card1, card2):
    mask = np.zeros((len(coefficients), card1, card2), dtype=int)
    num_k = (len(coefficients))
    #mod_second = len(coefficients)
    #mod_first = max(card1, mod_second)
    for k in range(len(coefficients)):
        for i in range(card1):
            for j in range(card2):
                #mask[k][i][j] = 1 if ((((i+j-k)%mod_first)%mod_second)==0) else 0
                 mask[k][i][j] = 1 if ((i+j-k))%num_k == 0 else 0
    
    #print(mask)
    #print(coefficients[:, np.newaxis, np.newaxis])
    #print(mask * coefficients[:, np.newaxis, np.newaxis])
    
    return np.sum(mask * coefficients[:, np.newaxis, np.newaxis], axis=0)

def create_rotation_mask(num_coeffs, weight_shape, dtype=tf.float64):
    """Create rotation mask for given number coefficients and weight matrix shape"""
    num_rows = weight_shape[-2]
    num_cols = weight_shape[-1]
    # Init rotation mask
    mask = np.zeros((num_coeffs, num_rows, num_cols), dtype=np.uint8)
    # Fill rotation mask
    for k in range(num_coeffs):
        for i in range(num_rows):
            for j in range(num_cols):
                mask[k,i,j] = 1 if ((i+j-k))%num_coeffs == 0 else 0
    mask = tf.constant(mask, dtype=dtype)
    
    return mask

def generate_weights1(coefficients, weight_shape):
    # Init rotation mask
    num_koeffs = coefficients.shape[-1]
    num_rows = weight_shape[-2]
    num_cols = weight_shape[-1]
    mask = np.zeros((num_koeffs, num_rows, num_cols), dtype=np.uint8)
    # Fill rotation mask
    for k in range(num_koeffs):
        for i in range(num_rows):
            for j in range(num_cols):
                mask[k,i,j] = 1 if ((i+j-k))%num_koeffs == 0 else 0
    mask = tf.constant(mask, dtype=coefficients.dtype)
    # Convert coefficients to matrices
    coeffs_reshaped = tf.reshape(
        coefficients, 
        tf.concat([tf.ones([weight_shape.shape[0]-2], tf.int32), coefficients.shape], 0)
    )
    coefficients = tf.tile(
        coeffs_reshaped, 
        tf.concat([weight_shape[:-2], [1]], 0)
    )
    coeffs_transformed = tf.reshape(coefficients, tf.concat([coefficients.shape, (1, 1)], 0))
    return tf.reduce_sum(mask * coeffs_transformed, axis=tf.rank(coeffs_transformed)-3)

def coeffs_to_weight_gen(coeffs, weight_shape, rotation_mask=None):
    """Create weigth generator that returns shared weight matrix. 
    At the lower depth level weights are coefficient rotations.
    The same rotation matrix is repeated over all units. 
    """
    # Create rotation mask
    if rotation_mask is None:
        rotation_mask = create_rotation_mask(coeffs.shape[-1], weight_shape)
    # Create weight generator
    def weight_gen():
        nonlocal coeffs
        nonlocal weight_shape
        nonlocal rotation_mask
        # Expand dimensions of coefficient matrix to match desired weight shape
        coeffs_shape = tf.concat([tf.ones([weight_shape.shape[0]-2], tf.int32), coeffs.shape, [1, 1]], 0)
        coeffs_reshaped = tf.reshape(coeffs, coeffs_shape)
        # Fill coefficient matrix with repeating tiles to match desired weight shape 
        coeffs_tiles = tf.concat([weight_shape[:-2], [1, 1, 1]], 0)
        coeffs_tiled = tf.tile(coeffs_reshaped, coeffs_tiles)
        # Create rotation matrix
        return tf.reduce_sum(rotation_mask * coeffs_tiled, axis=tf.rank(coeffs_tiled)-3)
    
    return weight_gen

def outer_weights_to_weight_gen(outer_weights, weight_shape, output_units, input_item_rank):
    """Create weigth generator that returns shared weight matrix. 
    Output weights of each unit are used as coefficients for lower-depth-level rotation weight matrix.
    """
    # Weight shape at current depth level
    current_weight_shape = tf.concat([weight_shape[:input_item_rank], weight_shape[-1:]], 0)
    # Create rotation mask
    rotation_mask = create_rotation_mask(output_units, current_weight_shape)
    # Create weight generator
    def weight_gen():
        nonlocal outer_weights
        nonlocal rotation_mask
        # Get outer weights
        outer_weights_inner = outer_weights() if callable(outer_weights) else outer_weights
        # Spread outer weights 
        outer_weights_spread = tf.reshape(outer_weights_inner, tf.concat([outer_weights_inner.shape, (1, 1)], 0))
        # Create rotation matrix
        return tf.reduce_sum(rotation_mask * outer_weights_spread, axis=tf.rank(outer_weights_spread)-3)
    
    if current_weight_shape.shape[0] != weight_shape.shape[0]:
        # Add lower-depth level weight generator
        weight_gen = outer_weights_to_weight_gen(weight_gen, weight_shape, output_units, input_item_rank+1) 
    
    return weight_gen

def outer_weights_to_weight_gen(outer_weights, weight_shape, output_units):
    """Create weigth generator that returns shared weight matrix. 
    Output weights of each unit are used as coefficients for lower-depth-level rotation weight matrix.
    """
    # Create rotation mask
    rotation_mask = create_rotation_mask(output_units, weight_shape)
    # Create weight generator
    def weight_gen():
        nonlocal outer_weights
        nonlocal rotation_mask
        # Get outer weights
        outer_weights_inner = outer_weights() if callable(outer_weights) else outer_weights
        # Spread outer weights 
        outer_weights_spread = tf.reshape(outer_weights_inner, tf.concat([outer_weights_inner.shape, (1, 1)], 0))
        # Create rotation matrix
        return tf.reduce_sum(rotation_mask * outer_weights_spread, axis=tf.rank(outer_weights_spread)-3)
    
    return weight_gen

def generate_weights2(outer_weights, weight_shape): # outer weights without bias
    # Init rotation mask
    num_koeffs = outer_weights.shape[-1]
    num_rows = weight_shape[-2]
    num_cols = weight_shape[-1]
    mask = np.zeros((num_koeffs, num_rows, num_cols), dtype=np.uint8)
    # Fill rotation mask
    for k in range(num_koeffs):
        for i in range(num_rows):
            for j in range(num_cols):
                mask[k,i,j] = 1 if ((i+j-k))%num_koeffs == 0 else 0
    mask = tf.constant(mask, dtype=outer_weights.dtype)
    # Spread outer weights 
    outer_weights_transformed = tf.reshape(outer_weights, tf.concat([outer_weights.shape, (1, 1)], 0))
    return tf.reduce_sum(mask * outer_weights_transformed, axis=tf.rank(outer_weights_transformed)-3)

def outer_weights_to_weight_gen(outer_weights, weight_shape, output_units, input_item_rank):
    """Create weigth generator that returns shared weight matrix. 
    Output weights of each unit are used as coefficients for lower-depth-level rotation weight matrix.
    """
    # Weight shape at current depth level
    current_weight_shape = tf.concat([weight_shape[:input_item_rank], weight_shape[-1:]], 0)
    # Create rotation mask
    rotation_mask = create_rotation_mask(output_units, current_weight_shape)
    # Create weight generator
    def weight_gen():
        nonlocal outer_weights
        nonlocal rotation_mask
        # Get outer weights
        outer_weights_inner = outer_weights() if callable(outer_weights) else outer_weights
        # Spread outer weights 
        outer_weights_spread = tf.reshape(outer_weights_inner, tf.concat([outer_weights_inner.shape, (1, 1)], 0))
        # Create rotation matrix
        return tf.reduce_sum(rotation_mask * outer_weights_spread, axis=tf.rank(outer_weights_spread)-3)
    
    if current_weight_shape.shape[0] != weight_shape.shape[0]:
        # Add lower-depth level weight generator
        weight_gen = outer_weights_to_weight_gen(weight_gen, weight_shape, output_units, input_item_rank+1) 
    
    return weight_gen

In [50]:
input_num = 4
hidden_num = 4
output_num = 2

inner_units = 2

input_item_rank = 2


#coefficients = 0.1 * (np.arange(4, dtype=float) + 1)
#print(generate_weights(coefficients, hidden_num + 1, output_num))

weight_shape = tf.concat([[input_num, hidden_num], tf.fill([input_item_rank-1], inner_units)], 0)
coefficients = 0.1 * (tf.range(weight_shape[-1], dtype=tf.float64) + 1)
# coeffs_reshaped = tf.reshape(
#     coefficients, 
#     tf.concat([tf.ones([input_item_rank], tf.int32), coefficients.shape], 0)
# )
# coefficients = tf.tile(
#     coeffs_reshaped, 
#     tf.concat([tf.fill([input_item_rank], hidden_num), [1]], 0)
# )
# print(coefficients)
print(generate_weights1(coefficients, weight_shape))

tf.Tensor(
[[[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]

 [[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]

 [[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]

 [[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]], shape=(4, 4, 2), dtype=float64)


In [51]:
rotation_mask = create_rotation_mask(coefficients.shape[0], weight_shape)
weight_gen = coeffs_to_weight_gen(coefficients, weight_shape, rotation_mask)
print(weight_gen())

tf.Tensor(
[[[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]

 [[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]

 [[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]

 [[0.1 0.2]
  [0.2 0.1]
  [0.1 0.2]
  [0.2 0.1]]], shape=(4, 4, 2), dtype=float64)


In [22]:
input_num = 4
hidden_num = 3
output_num = 4

inner_units = 2

weights = 0.1 * (tf.range((hidden_num + 1) * output_num, dtype=tf.float64) + 1)
weights = tf.reshape(weights, (hidden_num+1, output_num))
# print(weights.shape)
print(weights)
weights = generate_weights2(weights[...,:-1,:], (hidden_num, inner_units+1, output_num))
# print(weights.shape)
print(weights)
# weights = generate_weights2(weights[...,:-1,:], (hidden_num, inner_units, inner_units+1, output_num))
# print(weights.shape)
# weights = generate_weights2(weights[...,:-1,:], (hidden_num, inner_units, inner_units, inner_units+1, output_num))
# print(weights.shape)

tf.Tensor(
[[0.1 0.2 0.3 0.4]
 [0.5 0.6 0.7 0.8]
 [0.9 1.  1.1 1.2]
 [1.3 1.4 1.5 1.6]], shape=(4, 4), dtype=float64)
tf.Tensor(
[[[0.1 0.2 0.3 0.4]
  [0.2 0.3 0.4 0.1]
  [0.3 0.4 0.1 0.2]]

 [[0.5 0.6 0.7 0.8]
  [0.6 0.7 0.8 0.5]
  [0.7 0.8 0.5 0.6]]

 [[0.9 1.  1.1 1.2]
  [1.  1.1 1.2 0.9]
  [1.1 1.2 0.9 1. ]]], shape=(3, 3, 4), dtype=float64)


In [23]:
weights = 0.1 * (tf.range((hidden_num + 1) * output_num, dtype=tf.float64) + 1)
weights = tf.reshape(weights, (hidden_num+1, output_num))
print(weights)
weight_shape = tf.constant((hidden_num, inner_units+1, output_num))
weight_gen = outer_weights_to_weight_gen(weights[...,:-1,:], weight_shape, weights.shape[-1], 2)
# print(weight_gen().shape)
print(weight_gen())

# weight_shape = tf.constant((hidden_num, inner_units, inner_units+1, output_num))
# weight_gen = outer_weights_to_weight_gen(weights[...,:-1,:], weight_shape, weights.shape[-1], 2)
# print(weight_gen().shape)

# weight_shape = tf.constant((hidden_num, inner_units, inner_units, inner_units+1, output_num))
# weight_gen = outer_weights_to_weight_gen(weights[...,:-1,:], weight_shape, weights.shape[-1], 2)
# print(weight_gen().shape)

tf.Tensor(
[[0.1 0.2 0.3 0.4]
 [0.5 0.6 0.7 0.8]
 [0.9 1.  1.1 1.2]
 [1.3 1.4 1.5 1.6]], shape=(4, 4), dtype=float64)
tf.Tensor(
[[[0.1 0.2 0.3 0.4]
  [0.2 0.3 0.4 0.1]
  [0.3 0.4 0.1 0.2]]

 [[0.5 0.6 0.7 0.8]
  [0.6 0.7 0.8 0.5]
  [0.7 0.8 0.5 0.6]]

 [[0.9 1.  1.1 1.2]
  [1.  1.1 1.2 0.9]
  [1.1 1.2 0.9 1. ]]], shape=(3, 3, 4), dtype=float64)
